$\qquad$ $\qquad$$\qquad$  **TDA 231 Machine Learning: Homework 5** <br />
$\qquad$ $\qquad$$\qquad$ **Goal: Clustering**<br />
$\qquad$ $\qquad$$\qquad$                   **Grader: Aristide** <br />
$\qquad$ $\qquad$$\qquad$                     **Due Date: 21/5** <br />
$\qquad$ $\qquad$$\qquad$                   **Submitted by: Name, Personal no., email** <br />

General guidelines:
* Since there is no theoretical part for this assigment, submit this ipynb only (with completed code/results).
* All discussion regarding practical problems, along with solutions and plots should be specified here itself. We will not generate the solutions/plots again by running your code.
* Your name, personal number and email address should be specified above.
* All datasets can be downloaded from the course website.
* All plots/tables and other relevant information should be included.

# Practical problems

The follwing code might be useful for this excercise.

```python
import scipy.io
mat = scipy.io.loadmat('hw5_p1a.mat')
print (mat.keys())
X = mat['X']
```

## [K-Means Implementation, 20 points]

a. Implement the basic (linear) $k$-means algorithm as described in the lecture, using the euclidean distance. Use (uniformly) random points from the data as initialization for the centroids. Terminate the iterative procedure when the the cluster assignments do not change.

b. Run your implementation on the matrix $X$ in **hw5_p1a.mat** with $k=2$. Each row of the matrix is an observation, and each column is a feature. Store the cluster assignment both after 2 iterations, and at convergence. Produce a scatter plot of the data with colors indicating the cluster assignments at convergence and highlight points that have changed assignment after the second iteration.

c. Implement the kernel $k$-means algorithm as described in the lecture, using the Gaussian RBF-kernel.

d. Run the linear $k$-means **and** your kernel $k$-means on **hw5_p1b.mat** with $k=2$. For the Gaussian RBF-kernel, use $\sigma=0.2$. Produce scatter plots of the data, with color indicating the cluster assignment at convergence, one plot for each of the algorithms.

In [203]:
import scipy.io
import random
import numpy as np;
import matplotlib.pyplot as plt
mat = scipy.io.loadmat('hw5_p1a.mat')
X = mat['X']

def KMeans(k, data):
    means = [];
    oldMeans = [];
    assignments2Iterations = [];
    # Select K random means
    for i in range(0,k):
        means.append(data[random.randint(0, len(data))])
    oldMeans = means;
    iterations = 1;
    while(True):
        oldMeans = list(means);
        assignments = [[]]
        assignments.append([])

        for i in data:
            distancesToMeans = []
            for j in range(0,k):
                distancesToMeans.append((dist(i,means[j])))
                assignments[distancesToMeans.index(np.min(distancesToMeans))].append(i)
        
        for i in range(0,k):
            means[i] = np.mean(np.vstack(assignments[i]),axis=0)
        
        if np.allclose(oldMeans,means,0.0001):
            print("Breaking on iteration ", iterations)
            return assignments, assignments2Iterations, means;
        else: 
            iterations+=1;
        if iterations == 2:
            assignments2Iterations = list(assignments)
    
# Euclidian data
def dist(mu,x):
    return np.linalg.norm(np.array(mu)-np.array(x))
#B) 
convergence, assignment2Iterations, mean = KMeans(2,X);

# Sort points
cool = np.unique(convergence[0],assignment2Iterations[0],axis=1)
print(len(np.unique(np.vstack(convergence[0]),np.vstack(assignment2Iterations[0]),axis=0)))

plt.scatter(np.vstack(convergence[0])[:,0].T,np.vstack(convergence[0])[:,1].T,c='red',label="Class 1")
plt.scatter(np.vstack(convergence[1])[:,0].T,np.vstack(convergence[1])[:,1].T,c='blue', label="Class 2")

plt.scatter(np.vstack(mean[0])[:,0].T,np.vstack(mean[0])[:,0].T,c='green',s=100, label="Mean for class 1")
plt.scatter(np.vstack(mean[1]),np.vstack(mean[1]),c='yellow',s=100, label ="Mean for class 2")
plt.legend();



Breaking on iteration  7


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()